In [1]:
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et
import pandas as pd
import urllib.request
from shapely.geometry import Point, shape
import json
import math

In [2]:
city='Detroit'
SIM_AREA_PATH='./cities/'+city+'/clean/table_area.geojson'
SIM_NETWORK_PATH_ROOT='./cities/'+city+'/clean/'

sim_area=json.load(open(SIM_AREA_PATH))

full_area=[shape(f['geometry']) for f in sim_area['features']]
bounds=[shp.bounds for shp in full_area]
boundsAll=[min([b[0] for b in bounds]), #W
               min([b[1] for b in bounds]), #S
               max([b[2] for b in bounds]), #E
               max([b[3] for b in bounds])] #N

In [3]:
query='http://api.openstreetmap.org/api/0.6/map?bbox='+','.join([str(b) for b in boundsAll])
file = urllib.request.urlopen(query)
xData=et.parse(file)
file.close()

root=xData.getroot()
root.tag

nodeLat=[]
nodeLong=[]
nodeId=[]
aNodes=[]
bNodes=[]
hwTags=[]
names=[]
oneway=[]
ref=[]
maxspeed=[]
osmid=[]

for child in root:
    if child.tag=='node':
        nodeLat.extend([child.get('lat')])
        nodeLong.extend([child.get('lon')])
        nodeId.extend([child.get('id')])
    elif child.tag=='way':
        highwayTag, name, oneW, re, mSp=[], [], [], [], []
        for t in child.findall('tag'):
            key=t.get('k')
            if key=='highway':
                value=t.get('v')
                highwayTag.extend([value])
        #now confirmed if the child is a way: if so, loop through the tags again and save them
        if any(highwayTag):
            for t in child.findall('tag'):
                if t.get('k')=='name':
                    name=t.get('v')
                if t.get('k')=='oneway':
                    oneW=t.get('v')
                if t.get('k')=='ref':
                    re=t.get('v')
                if t.get('k')=='maxspeed':
                    mSp=t.get('v')
            first=1
            for gChild in child:
                if gChild.tag=='nd':
                    if first==1:
                        #aNodes.extend([gChild.get('ref')])
                        lastNode=gChild.get('ref')
                    else:
                        aNodes.extend([lastNode])
                        lastNode=gChild.get('ref')
                        bNodes.extend([lastNode])
                        hwTags.extend([highwayTag[0]])
                        names.extend([name])
                        oneway.extend([oneW])
                        ref.extend([re])
                        maxspeed.extend([mSp])
                        osmid.extend([list(child.getiterator())[0].attrib['id']])
                    first=0

In [4]:
latLong= [[nodeLat[i], nodeLong[i]] for i in range(len(nodeId))]
LatLongDict = dict(zip(nodeId, latLong))

aNodeLat=[LatLongDict[aNodes[i]][0] for i in range(len(aNodes))]
aNodeLon=[LatLongDict[aNodes[i]][1] for i in range(len(aNodes))]
bNodeLat=[LatLongDict[bNodes[i]][0] for i in range(len(bNodes))]
bNodeLon=[LatLongDict[bNodes[i]][1] for i in range(len(bNodes))]

network=pd.DataFrame({'aNodes':aNodes, 'bNodes':bNodes, 
'aNodeLat':aNodeLat, 'aNodeLon':aNodeLon,'bNodeLat':bNodeLat,'bNodeLon':bNodeLon, 'type':hwTags, 
'name':names, 'speed':maxspeed, 'ref':ref, 'osmid':osmid, 'oneway':oneway})

In [5]:
%matplotlib inline

In [6]:
from ipyleaflet import Map, Polyline, basemaps, basemap_to_tiles, Circle, Marker

In [7]:
polylines=[]
m = Map(center = (aNodeLat[0], aNodeLon[0]), zoom =14, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
lines=[]
for ind, row in network.iterrows():
    lines.append([[float(row['aNodeLat']), float(row['aNodeLon'])], [float(row['bNodeLat']), float(row['bNodeLon'])]])

lineLayer = Polyline(
    locations = lines,
    weight=1,
    opacity=1,
    color='white',
    fill=False
)
m.add_layer(lineLayer)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [8]:
network.head()

,aNodes,bNodes,aNodeLat,aNodeLon,bNodeLat,bNodeLon,type,name,speed,ref,osmid,oneway
0,62547437,62547439,42.3326578,-83.0620911,42.3330202,-83.0622765,motorway_link,[],[],[],8727497,yes
1,62547439,62547441,42.3330202,-83.0622765,42.3332980,-83.0624543,motorway_link,[],[],[],8727497,yes
2,62547441,62547447,42.3332980,-83.0624543,42.3334178,-83.0625246,motorway_link,[],[],[],8727497,yes
3,62574668,62574679,42.3332176,-83.0727618,42.3333486,-83.0721618,motorway_link,[],[],[],8727643,yes
4,62574679,62574682,42.3333486,-83.0721618,42.3333736,-83.0720108,motorway_link,[],[],[],8727643,yes


# Find subset of links where at least one node is in the simulation area

In [9]:
print(len(LatLongDict))
sim_area_nodes=set()
# check if each node in any sim area, if so add to list
for node_num in LatLongDict:
    for feat in sim_area['features']:
        if shape(feat['geometry']).contains(Point(
            float(LatLongDict[node_num][1]), 
            float(LatLongDict[node_num][0]))):
            sim_area_nodes.add(node_num)
len(sim_area_nodes)

13547


8568

In [10]:
print(len(network))
network=network.loc[((network['aNodes'].isin(sim_area_nodes))|
    (network['bNodes'].isin(sim_area_nodes)))]
print(len(network))

7048
4025


# Plot network inside sim area only

In [11]:
polylines=[]
m = Map(center = (aNodeLat[0], aNodeLon[0]), zoom =15, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
lines=[]
for ind, row in network.iterrows():
    lines.append([[float(row['aNodeLat']), float(row['aNodeLon'])], [float(row['bNodeLat']), float(row['bNodeLon'])]])

lineLayer = Polyline(
    locations = lines,
    weight=1,
    opacity=1,
    color='white',
    fill=False
)
m.add_layer(lineLayer)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [12]:
edges_types=set(network['type'])
edges_types

{'footway',
 'pedestrian',
 'primary',
 'residential',
 'secondary',
 'secondary_link',
 'service',
 'tertiary',
 'track',
 'unclassified'}

In [13]:
drive_types=[t for t in edges_types if t not in ['footway', 'path', 'steps', 'pedestrian', 'track']]
cycle_types=[t for t in edges_types if t not in [ 'steps']]
pt_types=edges_types
walk_types=edges_types

Create networks for each mode

In [14]:
drive_edges=network.loc[network['type'].isin(drive_types)]
walk_edges=network.loc[network['type'].isin(walk_types)]
pt_edges=network.loc[network['type'].isin(pt_types)]
cycle_edges=network.loc[network['type'].isin(cycle_types)]

In [15]:
len(drive_edges)

1945

Create Geojson

In [16]:
SPEEDS_MET_S={'driving':30/3.6,
        'cycling':15/3.6,
        'walking':4.8/3.6,
        'pt': 20/3.6}

def df_to_geojson(edges_df, net_type):
    features=[]
    for e_ind, e_row in edges_df.iterrows():
        from_node=e_row['aNodes']
        to_node=e_row['bNodes']
        edge_type=net_type
        coordinates=[[float(LatLongDict[from_node][1]), float(LatLongDict[from_node][0])],
                     [float(LatLongDict[to_node][1]), float(LatLongDict[to_node][0])]]
        features.append({"type": "Feature",
                         "properties": {'edge_type': edge_type},
                         'geometry': {
                                 "type": "LineString",
                                 'coordinates':coordinates },
                         })
    return {"type": "FeatureCollection",
            "crs": { "type": "name", "properties": { "name": "epsg:4326" } },
            "features": features}

In [17]:
drive_net_geo=df_to_geojson(drive_edges, 'driving')
cycle_net_geo=df_to_geojson(drive_edges, 'cycling')
walk_net_geo=df_to_geojson(drive_edges, 'walking')
pt_net_geo=df_to_geojson(drive_edges, 'pt')

In [18]:
json.dump(drive_net_geo, open(SIM_NETWORK_PATH_ROOT+'driving_net.geojson', 'w'))
json.dump(walk_net_geo, open(SIM_NETWORK_PATH_ROOT+'walking_net.geojson', 'w'))
json.dump(pt_net_geo, open(SIM_NETWORK_PATH_ROOT+'pt_net.geojson', 'w'))
json.dump(cycle_net_geo, open(SIM_NETWORK_PATH_ROOT+'cycling_net.geojson', 'w'))